In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data143046


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import os

print("train_data_length: {}".format(len(os.listdir("/home/aistudio/djddData/train/samples/"))))
print("test_data_length: {}".format(len(os.listdir("/home/aistudio/djddData/Validations/samples/"))))
print(os.listdir("/home/aistudio/djddData"))

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

train_data_length: 338
test_data_length: 10
['Validations', 'train']


In [6]:
import paddle
paddle.__version__

'2.2.2'

In [7]:
from paddle.io import Dataset
import os
from PIL import Image


class DJDDData(Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, root_dir, samples_dir, labels_dir, transform):
        """
        步骤二：实现构造函数，定义数据集大小
        """
        super(DJDDData, self).__init__()
        self.root_dir = root_dir
        self.samples_dir = samples_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.samples_list = os.listdir(os.path.join(self.root_dir, self.samples_dir))
        self.labels_list = os.listdir(os.path.join(self.root_dir, self.labels_dir))

        self.samples_list.sort()
        self.labels_list.sort()

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        sampele_name = self.samples_list[index]
        label_name = self.labels_list[index]
        sampele_item_path = os.path.join(self.root_dir, self.samples_dir, sampele_name)
        label_item_path = os.path.join(self.root_dir, self.labels_dir, label_name)

        sample = Image.open(sampele_item_path)
        label = Image.open(label_item_path)

        sample = self.transform(sample)
        label = self.transform(label)
        
        return sample, label

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        assert len(self.samples_list) == len(self.labels_list)
        return len(self.samples_list)

In [8]:
from paddle import nn
import paddle
from collections import OrderedDict


class DJDDNetWork(nn.Layer):
    def __init__(self, width=64, depth=16):
        super(DJDDNetWork, self).__init__()

        self.width = width
        self.depth = depth

        # downsampling
        self.down_sample = nn.Conv2D(3, 4, kernel_size=2, stride=2, padding=0)
  
        # mian layers
        self.main_layers = nn.Sequential(
            ("Conv_1", nn.Conv2D(4, 64, kernel_size=3, padding="SAME")),
            ("ReLu_1", nn.ReLU()),
        )
        for i in range(1, depth - 1):
            in_size = width
            out_size = width
            self.main_layers.add_sublayer("Conv_{}".format(i+1), nn.Conv2D(in_size, out_size, kernel_size=3, padding="SAME"))
            self.main_layers.add_sublayer("ReLu_{}".format(i+1), nn.ReLU())
        
        # residual
        self.residual = nn.Conv2D(width, 12, (1, 1))

        # upsample
        self.up_sample = nn.Conv2DTranspose(12, 3, (2, 2), stride=(2, 2))

        # final processing
        self.final_process = nn.Sequential(
            nn.Conv2D(6, width, 3, padding="SAME"),
            nn.ReLU(),
            nn.Conv2D(width, 3, 1, padding="SAME")
        )


    def forward(self, inputs):
        y1 = self.down_sample(inputs)
        y2 = self.main_layers(y1)
        y3 = self.residual(y2)
        y4 = self.up_sample(y3)
        y5 = paddle.concat(x=[inputs, y4], axis=1)
        y = self.final_process(y5)
        return y


In [ ]:
import paddle.vision.transforms as T
from paddle.io import DataLoader
import datetime
import time

paddle.device.set_device("gpu:0")
# ================ Data =================================
# prepare for training data
train_root_dir = "/home/aistudio/djddData/train/"
train_samples_dir = "samples"
train_labels_dir = "labels"

# prepare for test data
test_root_dir = "/home/aistudio/djddData/Validations/"
test_samples_dir = "samples"
test_labels_dir = "labels"

# set transform.Compose
transform = T.Compose([T.ToTensor()])

# training data
train_dataset = DJDDData(train_root_dir, train_samples_dir, train_labels_dir, transform)
print("len of data is: {}".format(train_dataset.__len__()))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# testing data
test_dataset = DJDDData(test_root_dir, test_samples_dir, test_labels_dir, transform)
print("len of data is: {}".format(test_dataset.__len__()))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)
# ============= end of data ===============================

# ==============training configuration ====================

# creata model
model = DJDDNetWork()
# set trainging epochs
epochs = 2000
# optimizer
opt = paddle.optimizer.Adam(parameters=model.parameters())
# set loss function
loss_func = paddle.nn.MSELoss()

# ==============end of training configuration =============

# ==============start tringing ============================

training_step = 0
start_time = time.time()
first_test_flag = True
save_model_loss = 0
for epoch in range(epochs):
    total_train_loss = 0
    # set training mode
    model.train()
    for index, data in enumerate(train_loader):
        training_step += 1
        samples, labels = data
        predict = model(samples)
        loss = loss_func(predict, labels)
        total_train_loss += loss
        loss.backward()
        opt.step()
        opt.clear_grad()
        if training_step % 10 == 0:
            print(
                    "--- the {}.th epoch    total training step:{} Loss:{}   used time:{}s    {} ---".format(
                    epoch+1, 
                    training_step, 
                    total_train_loss.item(),
                    (time.time()-start_time),
                    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                )
            )

    model.eval()
    total_test_loss = 0
    for index, test_batch_data in enumerate(test_loader):
        test_samples, test_labels = test_batch_data
        test_predict = model(test_samples)
        test_loss = loss_func(test_predict, test_labels)
        total_test_loss += test_loss
        if first_test_flag:
            first_test_flag = False
            save_model_loss = total_test_loss
        
        if total_test_loss <= save_model_loss:
            # save model
            print("save model")
            paddle.save(model.state_dict(), "DJDD.pdparams")
            paddle.save(opt.state_dict(), "adam.pdopt")
    print("loss on test data: {}".format(total_test_loss))


len of data is: 338
len of data is: 10


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 